In [1]:
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

sys.path.insert(1, '../metadl/baselines/sentive_ai')
from sentive.sentive_vision_network import sentive_vision_network

from sentive.sentive_brain import sentive_brain

root_ep = 'episode_'
suf_eptest = '_1.csv'
root_test= 'test_'

episode = np.zeros([1,5,28,28,2])
test = np.zeros([5, 28,28,2])
for i in range(5):
    episode[0][i][:,:,0] = np.loadtxt(root_ep+str(i)+suf_eptest)
    episode[0][i][:,:,1] = np.loadtxt(root_ep+str(i)+suf_eptest)

    test[i][:,:,0] = np.loadtxt(root_test+str(i)+suf_eptest)
    test[i][:,:,1] = np.loadtxt(root_test+str(i)+suf_eptest)


In [ ]:
sbrain = sentive_brain(episode,5)

In [ ]:
sbrain.nnet[0].nrn_tls.get_neuron_from_id(49)

In [ ]:
sbrain.nnet[0].nrn_tls.get_neuron_from_id(72)

In [ ]:
sbrain.nnet[0].nrn_tls.get_neuron_from_id(97)

In [ ]:
sbrain.nnet[0].show_receptive_field_id(98)

In [ ]:
sbrain.nnet[0].show_receptive_field_id(97)

In [ ]:
sbrain.nnet[0].show_nrn_path_by_id(97)